In [9]:
import pandas as pd
import os
from ast import literal_eval

In [103]:
dirSOM=r'D:\Sources\COSMOS_photoZ\COSMOS_v3.0\SOM\forArticle'
nameOfExperiment='ex020'
datasetNames={'Train':r'05_COSMOS_SpectrZ_Zlim_1p2_SOMinfo.csv',
              'Test':r'05_COSMOS_SpectrZ_Zlim_1p2_Test_SOMinfo.csv',
              'Deimos':r'04_COSMOS_DEIMOS_QfFilter_SOMinfo.csv'}
dirDatasets=os.path.join(dirSOM,nameOfExperiment,'datasets')

In [104]:
idCol='Seq'
specZ='specZ'
photoZ_ML='photoZ_ML'
photoZ_SED='photoZ_SED'
residML='resid_ML'
residSED='resid_SED'
residML_SED='residML_SED'

In [107]:
datasets={}
cellIDs=['cellID_TrainSOM']
for key,val in datasetNames.items():
    datasets[key]=pd.read_csv(os.path.join(dirDatasets,val))
    for cellID in cellIDs:
        datasets[key][cellID]=[literal_eval(x) for x in datasets[key][cellID]]
ddatasets={0:datasets}

for i in range(1,11):
    d={}
    mapID='_'+str(i).zfill(3)
    for key,val in datasetNames.items():
        key=key+mapID
        d[key]=pd.read_csv(os.path.join(dirDatasets,val.replace('.csv',mapID+'.csv')))
        for cellID in cellIDs:
            d[key][cellID]=[literal_eval(x) for x in d[key][cellID]]
    ddatasets[i]=d

In [109]:
TrainTotal=ddatasets[1]['Train_001']
for i in range(2,11):
    TrainTotal=TrainTotal.merge(ddatasets[i]['Train_'+str(i).zfill(3)][['Seq','specZ_outlCoeff',
                                            'photoZ_ML_outlCoeff','photoZ_SED_outlCoeff']],on='Seq',
                                      suffixes=['','_'+str(i).zfill(3)])
TrainTotal=TrainTotal.rename(columns={'specZ_outlCoeff':'specZ_outlCoeff_001',
                            'photoZ_ML_outlCoeff':'photoZ_ML_outlCoeff_001',
                            'photoZ_SED_outlCoeff':'photoZ_SED_outlCoeff_001'})

In [110]:
colsFilt=['specZ','photoZ_ML','photoZ_SED']
getMinValue = lambda l: l[l.abs().idxmin()]
getMedianValue = lambda l: l[l.abs().idxmin()]
for c in colsFilt:
    colsMerge=TrainTotal.columns
    colsMerge=[cc for cc in colsMerge if c in cc]
    TrainTotal[c+'_outlCoeff_Min']=TrainTotal[colsMerge].apply(getMinValue,axis=1)

In [112]:
TrainTotal[['specZ_outlCoeff_001','specZ_outlCoeff_002','specZ_outlCoeff_003','specZ_outlCoeff_Min']]

,specZ_outlCoeff_001,specZ_outlCoeff_002,specZ_outlCoeff_003,specZ_outlCoeff_Min
0,0.035780,0.459005,-0.440721,-0.013352
1,-0.028655,0.010800,0.539241,-0.004348
2,-0.110431,1.136810,0.759205,-0.053303
3,-0.548072,-0.903781,-0.574797,-0.017798
4,-0.446989,-0.568717,-0.796886,-0.025170
...,...,...,...,...
13917,-0.292942,-0.579148,-0.433558,-0.289654
13918,-1.114351,-1.834523,-1.451428,0.456574
13919,-0.758236,-0.649139,-0.469636,-0.019375
13920,0.375675,-0.464950,-0.514190,0.025834


In [113]:
TrainTotal.to_csv(os.path.join(dirDatasets,datasetNames['Train'].replace('.csv','_Total.csv')),index=False)